In [25]:
import os
import numpy as np
import pandas as pd

In [26]:
folder_path = '../pipeline/sim_lcs_output'

# List to hold individual DataFrames
data_frames = []

# Traverse the directory structure
for root, _, files in os.walk(folder_path):
    for file in files:
        if 'Perm' in file and file.endswith('.csv') and 'ExSTraCS' in file:
            file_path = os.path.join(root, file)
            # Read CSV into a DataFrame
            df = pd.read_csv(file_path, index_col='times')
            # Append to the list
            data_frames.append(df)
    
    # Concatenate all DataFrames
merged_df = pd.concat(data_frames, axis=1, join='outer').reset_index()
merged_df

,times,b_scores_het_f100_maf0.2_cens0.1perm8_cv3,b_scores_het_f100_maf0.2_cens0.1perm15_cv3,b_scores_het_f100_maf0.2_cens0.1perm5_cv3,b_scores_het_f100_maf0.2_cens0.1perm11_cv0,b_scores_het_f100_maf0.2_cens0.1perm12_cv3,b_scores_het_f100_maf0.2_cens0.1perm10_cv2,b_scores_het_f100_maf0.2_cens0.1perm1_cv4,b_scores_het_f100_maf0.2_cens0.1perm3_cv2,b_scores_het_f100_maf0.2_cens0.1perm15_cv4,...,b_scores_me_f100_maf0.4_cens0.4perm18_cv1,b_scores_me_f100_maf0.4_cens0.4perm19_cv2,b_scores_me_f100_maf0.4_cens0.4perm10_cv1,b_scores_me_f100_maf0.4_cens0.4perm9_cv2,b_scores_me_f100_maf0.4_cens0.4perm4_cv2,b_scores_me_f100_maf0.4_cens0.4perm16_cv1,b_scores_me_f100_maf0.4_cens0.4perm12_cv1,b_scores_me_f100_maf0.4_cens0.4perm7_cv3,b_scores_me_f100_maf0.4_cens0.4perm11_cv2,b_scores_me_f100_maf0.4_cens0.4perm7_cv0
0,1.0,0.554545,0.529734,0.551078,0.557874,0.529972,0.615522,0.536223,0.610651,0.535078,...,0.005000,NaN,0.005000,NaN,NaN,0.005006,0.005000,NaN,NaN,0.005000
1,2.0,0.428585,0.392532,0.428203,0.435880,0.409318,0.479316,0.415823,0.476050,0.416933,...,0.005000,NaN,0.004999,NaN,NaN,0.005006,0.005000,NaN,NaN,0.004999
2,3.0,0.337815,0.305886,0.339174,0.344723,0.323061,0.370284,0.329598,0.368161,0.331841,...,0.005000,0.005006,0.004999,0.005006,0.004999,0.005005,0.005000,NaN,0.005006,0.004998
3,4.0,0.296845,0.271888,0.299252,0.303390,0.284298,0.314578,0.289635,0.312609,0.291510,...,0.004999,0.005006,0.004998,0.005006,0.004999,0.005004,0.005000,NaN,0.005006,0.004997
4,5.0,0.292953,0.268900,0.295881,0.298952,0.281404,0.309547,0.286143,0.307676,0.288575,...,0.009999,0.005006,0.009994,0.005005,0.004999,0.010009,0.009999,NaN,0.005006,0.004995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,94.0,0.127211,0.164231,0.124007,0.134842,0.126135,0.125086,0.121815,0.126275,0.117402,...,0.062227,0.068596,0.042748,0.028188,0.067222,0.048107,0.035924,0.127099,0.048490,NaN
94,95.0,0.127211,0.164222,0.124004,0.127236,0.126135,0.125086,0.122869,0.126275,0.118333,...,0.062227,0.068595,0.042748,0.028188,0.067222,0.048108,0.035924,0.127100,0.048490,NaN
95,96.0,0.127211,0.164212,0.124001,0.123912,0.126135,NaN,0.122869,NaN,0.120660,...,0.062227,NaN,0.042748,NaN,NaN,0.072026,0.035924,0.253660,NaN,NaN
96,97.0,0.127211,0.164204,0.123999,NaN,0.126135,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.253662,NaN,NaN


In [27]:
def fun(df, col_name='b_scores_het_f100_maf0.4_cens0.8_perm6_cv1'):
    temp_df = df[[col_name, 'times']]
    temp_df = temp_df.dropna()
    try:
        val = np.trapz(temp_df[col_name], temp_df['times']) / (list(temp_df['times'])[-1] - list(temp_df['times'])[0])
    except Exception as e:
        display(temp_df)
        raise e
    return val

In [28]:
ibs_df = pd.DataFrame(columns=['Dataset', 'IBS'])
for col in merged_df.columns:
    if 'times' not in col:
        val = fun(merged_df, col)
        ibs_df = pd.concat([pd.DataFrame([[col,val]], columns=ibs_df.columns), ibs_df], ignore_index=True)
ibs_df = ibs_df.sort_values('Dataset').reset_index(drop=True)
ibs_df

,Dataset,IBS
0,b_scores_add_f100_maf0.2_cens0.1perm0_cv0,0.111393
1,b_scores_add_f100_maf0.2_cens0.1perm0_cv1,0.080668
2,b_scores_add_f100_maf0.2_cens0.1perm0_cv2,0.102585
3,b_scores_add_f100_maf0.2_cens0.1perm0_cv3,0.112587
4,b_scores_add_f100_maf0.2_cens0.1perm0_cv4,0.114964
...,...,...
2304,b_scores_me_f100_maf0.4_cens0.8perm9_cv0,0.576850
2305,b_scores_me_f100_maf0.4_cens0.8perm9_cv1,0.478994
2306,b_scores_me_f100_maf0.4_cens0.8perm9_cv2,0.518614
2307,b_scores_me_f100_maf0.4_cens0.8perm9_cv3,0.397383


In [29]:
ibs_df[['b', 'scores', 'model', 'nfeat', 'maf', 'cens', 'cv']] = pd.DataFrame(list(ibs_df['Dataset'].str.split('_')))
ibs_df.drop('Dataset', axis=1, inplace=True)
ibs_df.drop('scores', axis=1, inplace=True)
ibs_df.drop('b', axis=1, inplace=True)
ibs_df = ibs_df[['model', 'nfeat', 'maf', 'cens', 'cv', 'IBS']]
ibs_df

,model,nfeat,maf,cens,cv,IBS
0,add,f100,maf0.2,cens0.1perm0,cv0,0.111393
1,add,f100,maf0.2,cens0.1perm0,cv1,0.080668
2,add,f100,maf0.2,cens0.1perm0,cv2,0.102585
3,add,f100,maf0.2,cens0.1perm0,cv3,0.112587
4,add,f100,maf0.2,cens0.1perm0,cv4,0.114964
...,...,...,...,...,...,...
2304,me,f100,maf0.4,cens0.8perm9,cv0,0.576850
2305,me,f100,maf0.4,cens0.8perm9,cv1,0.478994
2306,me,f100,maf0.4,cens0.8perm9,cv2,0.518614
2307,me,f100,maf0.4,cens0.8perm9,cv3,0.397383


In [30]:
ibs_df['cens'] = ibs_df['cens'].str[:-5]

In [31]:
df = pd.read_csv('../pipeline/ibs_data_all.csv')
df = df.drop('times', axis=1)
df.columns = df.columns.str[9:]
df.columns = ['times'] + list(df.columns[1:])
org_ibs = df

In [32]:
def calc_perm_pvalue(ibs_df, model, maf, cens):
    temp_df = ibs_df[(ibs_df['model'] == model) & (ibs_df['maf']==maf) & (ibs_df['cens']==cens)]
    data_points = temp_df['IBS'].ravel()
    model_label = model+'_f100_'+maf+'_'+cens
    org_val = fun(org_ibs[[model_label, 'times']], model_label)
    pvalue = sum(data_points < org_val)/len(data_points)
    print(model, maf, cens, org_val, pvalue)
    return pvalue

In [33]:
models = [ 'me', 'add', 'epi', 'het']
mafs = ['maf0.2', 'maf0.4']
censs = ['cens0.1', 'cens0.4', 'cens0.8']

In [34]:
ibs_df

,model,nfeat,maf,cens,cv,IBS
0,add,f100,maf0.2,cens0.1,cv0,0.111393
1,add,f100,maf0.2,cens0.1,cv1,0.080668
2,add,f100,maf0.2,cens0.1,cv2,0.102585
3,add,f100,maf0.2,cens0.1,cv3,0.112587
4,add,f100,maf0.2,cens0.1,cv4,0.114964
...,...,...,...,...,...,...
2304,me,f100,maf0.4,cens0.8,cv0,0.576850
2305,me,f100,maf0.4,cens0.8,cv1,0.478994
2306,me,f100,maf0.4,cens0.8,cv2,0.518614
2307,me,f100,maf0.4,cens0.8,cv3,0.397383


In [35]:
pval_df = pd.DataFrame(columns=['Models', 'MAF', 'Cens', 'p-val'])
for model in models:
    for maf in mafs:
        for cens in censs:
            pval = calc_perm_pvalue(ibs_df, model, maf, cens)
            pval_df = pd.concat([pd.DataFrame([[model, maf, cens, pval]], columns=pval_df.columns), pval_df], ignore_index=True)
pval_df = pval_df.reindex(index=pval_df.index[::-1]).reset_index(drop=True)
pval_df

me maf0.2 cens0.1 0.12986549180107024 0.0
me maf0.2 cens0.4 0.16036363442932658 0.0
me maf0.2 cens0.8 0.3427810734319918 0.4
me maf0.4 cens0.1 0.13762225087799945 0.0
me maf0.4 cens0.4 0.16677384829438077 0.0
me maf0.4 cens0.8 0.3185247389763209 0.0
add maf0.2 cens0.1 0.07801520396549573 0.0
add maf0.2 cens0.4 0.1647135945470925 0.0
add maf0.2 cens0.8 0.1902257497138824 0.0
add maf0.4 cens0.1 0.0888027487524347 0.0
add maf0.4 cens0.4 0.17207601504345799 0.06
add maf0.4 cens0.8 0.3291777109610342 0.46
epi maf0.2 cens0.1 0.12638121019990337 0.0
epi maf0.2 cens0.4 0.11577496883182388 0.04
epi maf0.2 cens0.8 0.13313721920094598 0.2619047619047619
epi maf0.4 cens0.1 0.10278810341100571 0.02
epi maf0.4 cens0.4 0.1631831855538758 0.24
epi maf0.4 cens0.8 0.26742536318384547 0.3673469387755102
het maf0.2 cens0.1 0.17987884498719448 0.72
het maf0.2 cens0.4 0.12280347980787007 0.18
het maf0.2 cens0.8 0.22751418881702762 0.6904761904761905
het maf0.4 cens0.1 0.16174232825348514 0.2
het maf0.4 cens

,Models,MAF,Cens,p-val
0,me,maf0.2,cens0.1,0.000000
1,me,maf0.2,cens0.4,0.000000
2,me,maf0.2,cens0.8,0.400000
3,me,maf0.4,cens0.1,0.000000
4,me,maf0.4,cens0.4,0.000000
5,me,maf0.4,cens0.8,0.000000
6,add,maf0.2,cens0.1,0.000000
7,add,maf0.2,cens0.4,0.000000
8,add,maf0.2,cens0.8,0.000000
9,add,maf0.4,cens0.1,0.000000


In [36]:
pval_df[pval_df['p-val'] <= 0.05]

,Models,MAF,Cens,p-val
0,me,maf0.2,cens0.1,0.00
1,me,maf0.2,cens0.4,0.00
3,me,maf0.4,cens0.1,0.00
4,me,maf0.4,cens0.4,0.00
5,me,maf0.4,cens0.8,0.00
6,add,maf0.2,cens0.1,0.00
7,add,maf0.2,cens0.4,0.00
8,add,maf0.2,cens0.8,0.00
9,add,maf0.4,cens0.1,0.00
12,epi,maf0.2,cens0.1,0.00
